In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Add the final dbf file here

df_raw = pd.read_excel('Speed Trailer Work.xlsx', index_col = 'Group_ID')

In [3]:
df_raw

,Join_Count,TARGET_FID,JOIN_FID,Days,F_Requests,Ward_Num,PDFS,Seniors,Schools,Crashes
Group_ID,,,,,,,,,,
0,1,0,473,491,1,9,0,0,0,1
0,1,0,4010,491,1,9,0,0,0,1
1,1,1,3317,287,1,9,0,0,1,6
2,0,2,-1,280,1,9,0,0,1,0
3,1,3,3951,180,2,9,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
266,1,266,2430,280,4,7,0,0,0,1
267,0,267,-1,212,3,7,0,0,0,0
268,1,268,2417,60,2,7,0,0,0,1


In [4]:
# Add the raw input with all the requests information here

inputs = pd.read_excel('Inputs.xlsx')

In [5]:
# Add the Intersected dbf here after removing the duplicates

Intersected = pd.read_excel('Intersected.xlsx')

In [6]:
inputs.head()

,ID,Days since Request,Date,Name,Email,Phone Number,Neighbourhood,Detailed Location,Edited Location,City,Concat,Geocodes,Honored?
0,8,513,44377.0,Luisa Keen,jlkeen@shaw.ca,NaN,Ottewell,corner of 58 Street and 90 A Avenue eastbound,90A Avenue and 58 Street,"Edmonton, Alberta","90A Avenue and 58 Street Edmonton, Alberta","53.5246224, -113.4241492",Not Processed
1,10,510,44382.0,Teri Muhlbeier,terimuhlbeier@gmail.com,17807222070,Capilano,Along Capilano street between 59th Street and ...,Capilano Street and 63 Street,"Edmonton, Alberta","Capilano Street and 63 Street Edmonton, Alberta","53.5552175, -113.425397",False
2,15,506,44386.0,Frederick Hidalgo,poogzh@gmail.com,7807094863,The Hills at Charlesworth,Charlesworth Way and 5 Avenue intersection Cha...,5th Avenue and Charlesworth Way,"Edmonton, Alberta","5th Avenue and Charlesworth Way Edmonton, Alberta","53.4260791, -113.3995642",False
3,23,502,44392.0,Lindsay Vicente,lindsay.vicente@edmonton.ca,7808861189,Granville,On both sides of the road on Goodridge Link NW...,Goodridge Link and Goodridge Crescent,"Edmonton, Alberta",Goodridge Link and Goodridge Crescent Edmonton...,"53.5090254, -113.6801687",False
4,43,497,44399.0,Tim,timhastings.th@gmail.com,NaN,WestPointe of windermere,"Attached photo, leaving WestPointe on 170 st e...",Windermere Crescent and 170 Street,"Edmonton, Alberta","Windermere Crescent and 170 Street Edmonton, A...","53.44823, -113.61478",Not Processed


In [7]:
Intersected.head()

,FID_Full_B,Group_ID,Days,F_Requests,FID_Full_1,ID,Days_since
0,0,0,491,1,8,82,491
1,1,1,287,1,77,579,287
2,2,2,280,1,102,630,280
3,3,3,180,2,193,806,180
4,3,3,180,2,329,990,9


In [8]:
weights = {'Group_ID': 0 , 'F_Requests': 30, 'Seniors': 5, 'Schools': 10, 'Crashes': 25, 'Days': 30, 'Ward_Num': 0, 'PDFS': 0}

In [9]:
weights = pd.DataFrame.from_dict(weights, orient = 'index').T

In [10]:
weights.set_index('Group_ID', inplace = True)
weights

,F_Requests,Seniors,Schools,Crashes,Days,Ward_Num,PDFS
Group_ID,,,,,,,
0,30,5,10,25,30,0,0


In [11]:
# Groups in df_raw are repeated; so, the dataframe needs to be aggregated on a group level

df_raw.head()

,Join_Count,TARGET_FID,JOIN_FID,Days,F_Requests,Ward_Num,PDFS,Seniors,Schools,Crashes
Group_ID,,,,,,,,,,
0,1,0,473,491,1,9,0,0,0,1
0,1,0,4010,491,1,9,0,0,0,1
1,1,1,3317,287,1,9,0,0,1,6
2,0,2,-1,280,1,9,0,0,1,0
3,1,3,3951,180,2,9,0,0,0,1


In [12]:
# Creating a dataframe with the aggregate socres of the objectives

df = df_raw.pivot_table(index='Group_ID',values=['F_Requests', 'Days','Crashes','Schools','Seniors','PDFS', 'Ward_Num'],
                   aggfunc={'F_Requests': np.mean,'Days': np.mean,'Crashes':sum,'Schools':np.mean,'Seniors':np.mean,
                            'PDFS':np.mean, 'Ward_Num': np.mean})
df.head()

,Crashes,Days,F_Requests,PDFS,Schools,Seniors,Ward_Num
Group_ID,,,,,,,
0,2,491,1,0,0,0,9
1,6,287,1,0,1,0,9
2,0,280,1,0,1,0,9
3,1,180,2,0,0,0,9
4,0,223,1,0,0,0,2


In [13]:
df = df[weights.columns.tolist()]
df

,F_Requests,Seniors,Schools,Crashes,Days,Ward_Num,PDFS
Group_ID,,,,,,,
0,1,0,0,2,491,9,0
1,1,0,1,6,287,9,0
2,1,0,1,0,280,9,0
3,2,0,0,1,180,9,0
4,1,0,0,0,223,2,0
...,...,...,...,...,...,...,...
265,1,0,0,0,280,7,0
266,4,0,0,2,280,7,0
267,3,0,0,0,212,7,0


In [14]:
# Creating a dictionary of the 10 dataframes of the 10 wards considered (actually it creates 14 dataframe,
# but we are only interested in 10)

dfw = {} 
for i in range(1,13):
    dfw[i] = df[df['Ward_Num'] == i]
dfw[34] = df[(df['Ward_Num'] == 3) | (df['Ward_Num'] == 4)]
dfw[59] = df[(df['Ward_Num'] == 5) | (df['Ward_Num'] == 9)]

In [15]:
# dfc is a dictionary of the 10 dataframes containing the weights of the objectives of each group of requests

dfc = {}
for i in range(1,13):
    dfc[i] = dfw[i]/(dfw[i].max())
dfc[34] = dfw[34]/(dfw[34].max())
dfc[59] = dfw[59]/(dfw[59].max())

In [16]:
# Correcting the scores of schools and senior centers. If the group of request intersects with only 1 of the two objectives,
# the group will get the maximum score (instead of getting a percentage of the maximum number of schools/senior centers
# intersected)

for i in range(1,13):
    dfc[i][dfc[i][['Schools','Seniors']]>0] = 1 
    
    
dfc[34][dfc[34][['Schools','Seniors']]>0] = 1 
dfc[59][dfc[59][['Schools','Seniors']]>0] = 1 

In [17]:
# dfs is a dictionary of the 10 dataframes containing the scores of the objectives of each group of requests

dfs={}
for i in list(dfc.keys()):
    dfs[i] = dfc[i]*weights.values/weights.sum().sum()
    dfs[i] = dfs[i].fillna(0)

In [18]:
# Calculating the final scores of each group of requests in each of the 10 dataframes

for i in list(dfc.keys()):
    dfw[i]['Score'] = dfs[i]['F_Requests']+dfs[i]['Seniors']+dfs[i]['Schools']+dfs[i]['Crashes']+dfs[i]['Days']

<ipython-input-18-158f1d96c6ff>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfw[i]['Score'] = dfs[i]['F_Requests']+dfs[i]['Seniors']+dfs[i]['Schools']+dfs[i]['Crashes']+dfs[i]['Days']


In [19]:
dfw[2].sort_values(by ='Score').shape

(26, 8)

In [20]:
KEYS = list(dfc.keys())

In [21]:
KEYS.remove(3)
KEYS.remove(4)
KEYS.remove(5)
KEYS.remove(9)
KEYS

[1, 2, 6, 7, 8, 10, 11, 12, 34, 59]

In [22]:
# Creating a dataframe with the selected groups of requests based on their final scores

selected = pd.DataFrame(index = KEYS)
for i in KEYS:
    if dfw[i].empty:
        selected.loc[i,'Group_ID'] = -9999
        selected.loc[i,'Score'] = -9999
    else:    
        selected.loc[i,'Group_ID'] = (dfw[i]['Score'].idxmax())
        selected.loc[i,'Score'] = (dfw[i]['Score'].max())
selected['PDFS'] = list(df.loc[selected['Group_ID'], 'PDFS'])

In [23]:
# Dropping the unused wards

#selected.drop(labels = [3,4,5,9], axis = 0, inplace = True)

In [24]:
selected

,Group_ID,Score,PDFS
1,207.0,0.692272,0
2,26.0,0.693964,0
6,119.0,0.658460,0
7,257.0,0.576211,0
8,62.0,0.816531,0
10,136.0,0.598315,0
11,158.0,0.590914,0
12,261.0,0.714300,0
34,167.0,0.564137,0
59,21.0,0.657181,0


In [25]:
selected.sort_values('Group_ID')

,Group_ID,Score,PDFS
59,21.0,0.657181,0
2,26.0,0.693964,0
8,62.0,0.816531,0
6,119.0,0.658460,0
10,136.0,0.598315,0
11,158.0,0.590914,0
34,167.0,0.564137,0
1,207.0,0.692272,0
7,257.0,0.576211,0
12,261.0,0.714300,0


In [26]:
selected.index.names = ['Wards']

In [27]:
selected.to_excel('selected.xlsx', index = True);

In [28]:
selected_groups = list(selected['Group_ID'])

In [41]:
selected_groups

[207.0, 26.0, 119.0, 257.0, 62.0, 136.0, 158.0, 261.0, 204.0, 21.0]

In [42]:
selected_data = df[df.index.isin(selected_groups)][['Days','F_Requests','Schools','Seniors','Crashes','Ward_Num','PDFS']]
selected_data.style.background_gradient(cmap = 'Reds')

,Days,F_Requests,Schools,Seniors,Crashes,Ward_Num,PDFS
Group_ID,,,,,,,
21,490,2,0,0,4,5,0
26,487,1,0,0,4,2,0
62,272,3,2,0,7,8,0
119,13,2,1,0,3,6,0
136,394,2,2,0,4,10,0
158,107,2,1,0,2,11,0
204,29,3,1,0,4,4,0
207,170,3,3,1,4,1,0
257,279,2,0,0,10,7,0


In [43]:
Intersected = Intersected.drop_duplicates(subset='ID')

In [44]:
selected_groups

[207.0, 26.0, 119.0, 257.0, 62.0, 136.0, 158.0, 261.0, 204.0, 21.0]

In [45]:
#selected_groups = [207.0, 26.0, 119.0, 257.0, 62.0, 136.0, 158.0, 261.0, 204.0, 21.0]

In [46]:
req = Intersected[Intersected['Group_ID'].isin(selected_groups)][['Group_ID', 'ID']].merge(inputs, on = 'ID', how = 'left')
req

,Group_ID,ID,Days since Request,Date,Name,Email,Phone Number,Neighbourhood,Detailed Location,Edited Location,City,Concat,Geocodes,Honored?
0,21,119,490,44410.000000,Erin Schaefer,eringschaefer@gmail.com,5878797993,Summerside,STANTON Dr and 14 Ave sw Anywhere along Stanto...,Stanton Drive and 14 Avenue,"Edmonton, Alberta","Stanton Drive and 14 Avenue Edmonton, Alberta","53.421343, -113.453471",Not Processed
1,21,996,5,45089.438588,Hailey Glover,haileyglover200219@gmail.com,7802974991,Summerside,"Staton drive , 11 Ave and 76 street & Staton d...",11 Ave and Staton Dr NW,"Edmonton, Alberta","11 Ave and Staton Dr NW Edmonton, Alberta","53.423453, -113.455369",Not Processed
2,26,182,487,44413.000000,Heide,Hhorn644@mynorquest.ca,7809664676,Ambleside ( 1967 Ainslie Link ),By mailbox in Ainslie link,Ainslie Link and Allan Way,"Edmonton, Alberta","Ainslie Link and Allan Way Edmonton, Alberta","53.43033, -113.60291",Not Processed
3,62,659,272,44714.902477,Stephanie Dziedzic,stephaniedziedzic@live.com,NaN,Laurel,"18 avenue nw & 32 street, Details: 32 street b...",32 street and 23 avenue,"Edmonton, Alberta","32 street and 23 avenue Edmonton, Alberta","53.4539593, -113.3865392",False
4,62,959,21,45065.503692,Mandi Friesen (from engagement feedback for Si...,mandi.friesen@edmonton.ca,NaN,Silver Berry,No locations specified in feedback provided,Silver Berry Road and 29 St,"Edmonton, Alberta","Silver Berry Road and 29 St Edmonton, Alberta","53.457065, -113.382979",False
5,62,977,15,45075.414826,Helen Salga,medicinegal@gmail.com,NaN,Silver Berry,"25 Ave & 32 Street NW, Details:",25 Ave and 32 St NW,"Edmonton, Alberta","25 Ave and 32 St NW Edmonton, Alberta","53.455365, -113.386517",False
6,119,980,13,45077.587801,Amber Kate Engel,kateengel0@gmail.com,NaN,Allendale,"65 Avenue & 105 Street, Details:",65 Ave and 112 St NW,"Edmonton, Alberta","65 Ave and 112 St NW Edmonton, Alberta","53.502006, -113.520116",Not Processed
7,119,1007,2,45092.742917,NaN,aalessan@ualberta.ca,NaN,Parkallen,"67 Avenue & 112 A Street, Details:",67 Ave and 112 A St NW,"Edmonton, Alberta","67 Ave and 112 A St NW Edmonton, Alberta","53.503901, -113.521685",Not Processed
8,136,508,394,44544.325590,DAVE VYSE,Saferoads@edmonton.ca,NaN,Lynwood,83 avenue NW between 159 st nw and 156 st nw. ...,83 avenue and 157 street,"Edmonton, Alberta","83 avenue and 157 street Edmonton, Alberta","53.51643, -113.59348",False
9,136,787,195,44821.692882,Robert Mah,bobby2unes@gmail.com,7802033487,Lynnwood,"83 AV & 83 AV , Details: Too many vehicles an...",83 AVE & 156 st,"Edmonton, Alberta","83 AVE & 156 st Edmonton, Alberta","53.515794, -113.590357",Not Processed


In [47]:
req = req.sort_values('Group_ID')
req

,Group_ID,ID,Days since Request,Date,Name,Email,Phone Number,Neighbourhood,Detailed Location,Edited Location,City,Concat,Geocodes,Honored?
0,21,119,490,44410.000000,Erin Schaefer,eringschaefer@gmail.com,5878797993,Summerside,STANTON Dr and 14 Ave sw Anywhere along Stanto...,Stanton Drive and 14 Avenue,"Edmonton, Alberta","Stanton Drive and 14 Avenue Edmonton, Alberta","53.421343, -113.453471",Not Processed
1,21,996,5,45089.438588,Hailey Glover,haileyglover200219@gmail.com,7802974991,Summerside,"Staton drive , 11 Ave and 76 street & Staton d...",11 Ave and Staton Dr NW,"Edmonton, Alberta","11 Ave and Staton Dr NW Edmonton, Alberta","53.423453, -113.455369",Not Processed
2,26,182,487,44413.000000,Heide,Hhorn644@mynorquest.ca,7809664676,Ambleside ( 1967 Ainslie Link ),By mailbox in Ainslie link,Ainslie Link and Allan Way,"Edmonton, Alberta","Ainslie Link and Allan Way Edmonton, Alberta","53.43033, -113.60291",Not Processed
3,62,659,272,44714.902477,Stephanie Dziedzic,stephaniedziedzic@live.com,NaN,Laurel,"18 avenue nw & 32 street, Details: 32 street b...",32 street and 23 avenue,"Edmonton, Alberta","32 street and 23 avenue Edmonton, Alberta","53.4539593, -113.3865392",False
4,62,959,21,45065.503692,Mandi Friesen (from engagement feedback for Si...,mandi.friesen@edmonton.ca,NaN,Silver Berry,No locations specified in feedback provided,Silver Berry Road and 29 St,"Edmonton, Alberta","Silver Berry Road and 29 St Edmonton, Alberta","53.457065, -113.382979",False
5,62,977,15,45075.414826,Helen Salga,medicinegal@gmail.com,NaN,Silver Berry,"25 Ave & 32 Street NW, Details:",25 Ave and 32 St NW,"Edmonton, Alberta","25 Ave and 32 St NW Edmonton, Alberta","53.455365, -113.386517",False
6,119,980,13,45077.587801,Amber Kate Engel,kateengel0@gmail.com,NaN,Allendale,"65 Avenue & 105 Street, Details:",65 Ave and 112 St NW,"Edmonton, Alberta","65 Ave and 112 St NW Edmonton, Alberta","53.502006, -113.520116",Not Processed
7,119,1007,2,45092.742917,NaN,aalessan@ualberta.ca,NaN,Parkallen,"67 Avenue & 112 A Street, Details:",67 Ave and 112 A St NW,"Edmonton, Alberta","67 Ave and 112 A St NW Edmonton, Alberta","53.503901, -113.521685",Not Processed
8,136,508,394,44544.325590,DAVE VYSE,Saferoads@edmonton.ca,NaN,Lynwood,83 avenue NW between 159 st nw and 156 st nw. ...,83 avenue and 157 street,"Edmonton, Alberta","83 avenue and 157 street Edmonton, Alberta","53.51643, -113.59348",False
9,136,787,195,44821.692882,Robert Mah,bobby2unes@gmail.com,7802033487,Lynnwood,"83 AV & 83 AV , Details: Too many vehicles an...",83 AVE & 156 st,"Edmonton, Alberta","83 AVE & 156 st Edmonton, Alberta","53.515794, -113.590357",Not Processed


In [48]:
req.to_excel('Requests List.xlsx', index = False)

In [49]:
# View the scores of a particular Ward

dfw[2].sort_values('Score', ascending = False)

,F_Requests,Seniors,Schools,Crashes,Days,Ward_Num,PDFS,Score
Group_ID,,,,,,,,
26,1,0,0,4,487,2,0,0.693964
51,2,0,0,1,497,2,0,0.662500
73,2,0,0,0,490,2,0,0.595775
14,2,0,0,0,460,2,0,0.577666
43,1,0,0,4,242,2,0,0.546076
107,1,0,2,0,479,2,0,0.539135
36,1,0,0,3,318,2,0,0.529452
42,1,0,1,2,244,2,0,0.522284
13,2,0,1,0,178,2,0,0.507445


In [38]:
Intersected[Intersected['Group_ID'] == 180][['Group_ID', 'ID']].merge(inputs, on = 'ID', how = 'left')

,Group_ID,ID,Days since Request,Date,Name,Email,Phone Number,Neighbourhood,Detailed Location,Edited Location,City,Concat,Geocodes,Honored?
0,180,620,280,44702.726192,Heather McLeod,heathermcleod27@gmail.com,7802336484,Gold Bar,"106 Avenue & 42 Street, Details:",106 Ave and 44 street,"Edmonton, Alberta","106 Ave and 44 street Edmonton, Alberta","53.55104, -113.40702",False
